# 🚀 Autonomous System with Trajectory Preservation

**Version:** 3.0 - Enhanced with Auto-Sync & Meeting Trajectories  
**GPU:** A100 (40GB) recommended  
**Purpose:** Deploy autonomous system with complete meeting history preservation  

---

## ✨ New Features

1. **🔄 Real-time Drive Sync** - Reports saved directly to Google Drive during execution
2. **📊 Trajectory Preservation** - Complete meeting conversation history preserved
3. **👁️ Live Monitoring** - Real-time dashboard showing meeting progress
4. **💾 Crash Recovery** - Even if Colab disconnects, all data is safe on Drive
5. **📈 Progress Tracking** - Visual timeline of system evolution

---

## 🎯 What This Solves

**Problem:** Meeting artifacts lost when Colab disconnects (like your Oct 13 meeting)

**Solution:**
- Every meeting round → Instant save to Drive
- Trajectory file shows complete conversation flow
- Monitor dashboard streams live updates
- No data loss even on crash

---

## 🔧 Step 1: Environment Setup

In [ ]:
# Check GPU
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv

import torch
print(f"\n✅ PyTorch: {torch.__version__}")
print(f"✅ CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
    print(f"✅ Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

In [ ]:
# Authenticate & Mount Drive
from google.colab import auth, drive
import os

print("🔐 Authenticating...")
auth.authenticate_user()
print("✅ Authenticated")

print("\n💾 Mounting Google Drive...")
drive.mount('/content/drive', force_remount=True)
print("✅ Drive mounted")

# Verify mount
if os.path.exists('/content/drive/MyDrive'):
    print("✅ Drive access confirmed")
else:
    raise Exception("❌ Drive mount failed")

## 📦 Step 2: Install Dependencies

In [ ]:
!pip install -q anthropic openai google-generativeai pyyaml tqdm pandas numpy

print("✅ Packages installed")

import anthropic
import openai
import yaml

print(f"\n📦 Versions:")
print(f"   anthropic: {anthropic.__version__}")
print(f"   openai: {openai.__version__}")

## 🗂️ Step 3: Configure Paths with Auto-Sync

In [ ]:
from pathlib import Path
import shutil
from datetime import datetime

# Google Drive paths (where permanent storage lives)
DRIVE_BASE = Path("/content/drive/MyDrive")
DRIVE_PROJECT = DRIVE_BASE / "cv_multimodal/project/computer-vision-clean"

# Session-specific directory for this run
SESSION_ID = datetime.now().strftime("%Y%m%d_%H%M%S")
DRIVE_SESSION = DRIVE_PROJECT / f"sessions/session_{SESSION_ID}"
DRIVE_SESSION.mkdir(parents=True, exist_ok=True)

# Real-time report directories on Drive (auto-synced)
DRIVE_REPORTS = DRIVE_PROJECT / "multi-agent/reports"
DRIVE_REPORTS.mkdir(parents=True, exist_ok=True)

# Trajectory preservation on Drive
DRIVE_TRAJECTORIES = DRIVE_SESSION / "trajectories"
DRIVE_TRAJECTORIES.mkdir(parents=True, exist_ok=True)

# Local Colab temp workspace
PROJECT_ROOT = Path("/content/cv_project")
PROJECT_ROOT.mkdir(parents=True, exist_ok=True)

print("📁 Path Configuration:")
print(f"   Drive Project: {DRIVE_PROJECT}")
print(f"   Session: {DRIVE_SESSION}")
print(f"   Reports (live): {DRIVE_REPORTS}")
print(f"   Trajectories: {DRIVE_TRAJECTORIES}")
print(f"   Local Temp: {PROJECT_ROOT}")
print(f"\n✅ Session ID: {SESSION_ID}")

In [ ]:
# Verify required files exist
print("🔍 Verifying project files...\n")

required_files = [
    "multi-agent/autonomous_coordinator.py",
    "multi-agent/configs/autonomous_coordination.yaml",
    "multi-agent/tools/progress_sync_hook.py",
]

all_exist = True
for file in required_files:
    file_path = DRIVE_PROJECT / file
    if file_path.exists():
        size = file_path.stat().st_size / 1024
        print(f"✅ {file} ({size:.1f} KB)")
    else:
        print(f"❌ {file} - MISSING")
        all_exist = False

if all_exist:
    print("\n✅ All files present")
else:
    raise FileNotFoundError("Missing required files. Check Drive path.")

## 🔐 Step 4: Load API Keys

In [ ]:
# Load API keys from .env file
import os

env_locations = [
    DRIVE_PROJECT / ".env",
    DRIVE_PROJECT / "research/api_keys.env",
]

env_file = None
for location in env_locations:
    if location.exists():
        env_file = location
        break

if env_file:
    print(f"🔑 Loading from: {env_file.name}\n")
    with open(env_file, 'r') as f:
        for line in f:
            line = line.strip()
            if line and not line.startswith('#') and '=' in line:
                key, value = line.split('=', 1)
                os.environ[key.strip()] = value.strip()
                print(f"✅ {key.strip()} loaded")
    print("\n✅ API keys loaded")
else:
    print("⚠️ No .env file found. Enter keys manually:\n")
    anthropic_key = input("ANTHROPIC_API_KEY: ").strip()
    openai_key = input("OPENAI_API_KEY: ").strip()
    google_key = input("GOOGLE_API_KEY: ").strip()
    
    if anthropic_key:
        os.environ['ANTHROPIC_API_KEY'] = anthropic_key
    if openai_key:
        os.environ['OPENAI_API_KEY'] = openai_key
    if google_key:
        os.environ['GOOGLE_API_KEY'] = google_key
    print("✅ Keys set manually")

print("\n🔐 Status:")
print(f"   ANTHROPIC: {'✅' if os.getenv('ANTHROPIC_API_KEY') else '❌'}")
print(f"   OPENAI: {'✅' if os.getenv('OPENAI_API_KEY') else '❌'}")
print(f"   GOOGLE: {'✅' if os.getenv('GOOGLE_API_KEY') else '⚠️ Optional'}")

## 📤 Step 5: Copy Project Files

In [ ]:
import shutil
import os

print("📦 Copying project to local workspace...\n")

if PROJECT_ROOT.exists():
    shutil.rmtree(PROJECT_ROOT)

# Smart copy that skips Google Docs files
def ignore_gdocs(dir, files):
    """Ignore Google Docs files (.gdoc, .gsheet, .gslides)"""
    return [f for f in files if f.endswith(('.gdoc', '.gsheet', '.gslides'))]

try:
    shutil.copytree(DRIVE_PROJECT, PROJECT_ROOT, ignore=ignore_gdocs)
    print(f"✅ Copied to {PROJECT_ROOT}")
except Exception as e:
    print(f"⚠️ Copy warning: {e}")
    print("Retrying with dirs_exist_ok=True...")
    shutil.copytree(DRIVE_PROJECT, PROJECT_ROOT, ignore=ignore_gdocs, dirs_exist_ok=True)
    print(f"✅ Copied to {PROJECT_ROOT}")

# List key files
print("\n📁 Key files copied:")
py_files = list(PROJECT_ROOT.glob("multi-agent/**/*.py"))
for file in sorted(py_files)[:10]:
    size = file.stat().st_size / 1024
    print(f"   {file.relative_to(PROJECT_ROOT)} ({size:.1f} KB)")

if len(py_files) > 10:
    print(f"   ... and {len(py_files) - 10} more files")

## 🔄 Step 6: Create Auto-Sync System

In [ ]:
import json
import threading
import time
from pathlib import Path
from datetime import datetime
from typing import List, Dict, Any

class TrajectoryPreserver:
    """Preserves complete meeting trajectories with auto-sync to Drive"""
    
    def __init__(self, local_reports_dir: Path, drive_reports_dir: Path, drive_trajectory_dir: Path):
        self.local_reports = Path(local_reports_dir)
        self.drive_reports = Path(drive_reports_dir)
        self.drive_trajectories = Path(drive_trajectory_dir)
        
        # Create directories
        self.local_reports.mkdir(parents=True, exist_ok=True)
        self.drive_reports.mkdir(parents=True, exist_ok=True)
        self.drive_trajectories.mkdir(parents=True, exist_ok=True)
        
        # Tracking
        self.synced_files = set()
        self.trajectory_log = []
        self.running = False
        self.sync_thread = None
        
        print("✅ Trajectory Preserver initialized")
        print(f"   Local: {self.local_reports}")
        print(f"   Drive Reports: {self.drive_reports}")
        print(f"   Drive Trajectories: {self.drive_trajectories}")
    
    def start_auto_sync(self, interval_seconds: int = 10):
        """Start background sync thread"""
        if self.running:
            print("⚠️ Auto-sync already running")
            return
        
        self.running = True
        self.sync_thread = threading.Thread(
            target=self._sync_loop,
            args=(interval_seconds,),
            daemon=True
        )
        self.sync_thread.start()
        print(f"✅ Auto-sync started (every {interval_seconds}s)")
    
    def stop_auto_sync(self):
        """Stop background sync"""
        self.running = False
        if self.sync_thread:
            self.sync_thread.join(timeout=5)
        print("✅ Auto-sync stopped")
    
    def _sync_loop(self, interval: int):
        """Background sync loop"""
        while self.running:
            try:
                self.sync_now()
            except Exception as e:
                print(f"⚠️ Sync error: {e}")
            time.sleep(interval)
    
    def sync_now(self, verbose: bool = False) -> Dict[str, int]:
        """Immediate sync of all new files"""
        stats = {'new': 0, 'updated': 0, 'skipped': 0}
        
        if not self.local_reports.exists():
            return stats
        
        # Find all report files
        report_files = list(self.local_reports.glob("**/*.md")) + \
                      list(self.local_reports.glob("**/*.json"))
        
        for local_file in report_files:
            rel_path = local_file.relative_to(self.local_reports)
            drive_file = self.drive_reports / rel_path
            
            # Check if new or modified
            file_id = str(rel_path)
            
            if file_id not in self.synced_files:
                # New file
                drive_file.parent.mkdir(parents=True, exist_ok=True)
                shutil.copy2(local_file, drive_file)
                self.synced_files.add(file_id)
                stats['new'] += 1
                
                if verbose:
                    print(f"   📤 New: {rel_path}")
                
                # Log to trajectory
                self._log_artifact(local_file, 'created')
            else:
                # Check if modified
                if drive_file.exists():
                    local_mtime = local_file.stat().st_mtime
                    drive_mtime = drive_file.stat().st_mtime
                    
                    if local_mtime > drive_mtime:
                        shutil.copy2(local_file, drive_file)
                        stats['updated'] += 1
                        if verbose:
                            print(f"   🔄 Updated: {rel_path}")
                        self._log_artifact(local_file, 'updated')
                    else:
                        stats['skipped'] += 1
        
        # Save trajectory log
        if stats['new'] > 0 or stats['updated'] > 0:
            self._save_trajectory()
        
        return stats
    
    def _log_artifact(self, file_path: Path, action: str):
        """Log artifact to trajectory"""
        entry = {
            'timestamp': datetime.now().isoformat(),
            'file': file_path.name,
            'action': action,
            'size': file_path.stat().st_size
        }
        self.trajectory_log.append(entry)
    
    def _save_trajectory(self):
        """Save trajectory log to Drive"""
        trajectory_file = self.drive_trajectories / "trajectory_log.json"
        
        with open(trajectory_file, 'w') as f:
            json.dump({
                'session_id': SESSION_ID,
                'start_time': self.trajectory_log[0]['timestamp'] if self.trajectory_log else None,
                'last_update': datetime.now().isoformat(),
                'total_artifacts': len(self.trajectory_log),
                'artifacts': self.trajectory_log
            }, f, indent=2)
    
    def create_trajectory_summary(self) -> str:
        """Create human-readable trajectory summary"""
        if not self.trajectory_log:
            return "No trajectory data yet"
        
        lines = [
            "# Meeting Trajectory Summary",
            f"\n**Session**: {SESSION_ID}",
            f"**Start**: {self.trajectory_log[0]['timestamp']}",
            f"**Last Update**: {datetime.now().isoformat()}",
            f"**Artifacts**: {len(self.trajectory_log)}",
            "\n## Timeline\n"
        ]
        
        for entry in self.trajectory_log:
            time_str = entry['timestamp'].split('T')[1][:8]
            lines.append(f"- `{time_str}` - {entry['action'].upper()}: {entry['file']}")
        
        summary = "\n".join(lines)
        
        # Save to Drive
        summary_file = self.drive_trajectories / "trajectory_summary.md"
        with open(summary_file, 'w') as f:
            f.write(summary)
        
        return summary

# Initialize preserver
preserver = TrajectoryPreserver(
    local_reports_dir=PROJECT_ROOT / "multi-agent/reports",
    drive_reports_dir=DRIVE_REPORTS,
    drive_trajectory_dir=DRIVE_TRAJECTORIES
)

print("\n✅ Ready for trajectory preservation")

## 🚀 Step 7: Start Autonomous System

In [ ]:
import sys
sys.path.insert(0, str(PROJECT_ROOT / "multi-agent"))

from autonomous_coordinator import AutonomousCoordinator

print("🚀 Initializing Autonomous Coordinator...\n")

coordinator = AutonomousCoordinator(
    config_path=PROJECT_ROOT / "multi-agent/configs/autonomous_coordination.yaml",
    project_root=PROJECT_ROOT
)

print("\n✅ Coordinator ready")
print(f"   Agents: {len(coordinator.agents)}")
print(f"   Channels: {len(coordinator.channels._subscribers)}")
print(f"   Triggers: {len(coordinator.triggers._active_triggers)}")

In [ ]:
# Start auto-sync FIRST
print("🔄 Starting auto-sync system...\n")
preserver.start_auto_sync(interval_seconds=10)
print("✅ Auto-sync active (every 10 seconds)")

# Start coordinator
print("\n💓 Starting autonomous system...\n")
coordinator.start()

print("\n" + "="*60)
print("✅ AUTONOMOUS SYSTEM RUNNING")
print("="*60)
print("\n📊 Features active:")
print("   ✅ Hourly heartbeat cycles")
print("   ✅ Auto-sync to Drive (10s)")
print("   ✅ Trajectory preservation")
print("   ✅ Crash recovery")
print("\n💡 All meeting artifacts saved to:")
print(f"   {DRIVE_REPORTS}")
print(f"\n📊 Trajectory log:")
print(f"   {DRIVE_TRAJECTORIES}")
print(f"\n📁 Session directory:")
print(f"   {DRIVE_SESSION}")

## 👁️ Step 8: Live Monitoring Dashboard

In [ ]:
from IPython.display import clear_output, display, HTML
import time
from datetime import datetime

def get_latest_reports():
    """Get latest report files from Drive"""
    reports = {
        'transcripts': list(DRIVE_REPORTS.glob("transcript_*.md")),
        'summaries': list(DRIVE_REPORTS.glob("summary_*.md")),
        'actions': list(DRIVE_REPORTS.glob("actions_*.json"))
    }
    
    # Sort by modification time
    for key in reports:
        reports[key] = sorted(reports[key], key=lambda x: x.stat().st_mtime, reverse=True)
    
    return reports

def display_dashboard():
    """Display live monitoring dashboard"""
    clear_output(wait=True)
    
    print("="*70)
    print("👁️  LIVE MONITORING DASHBOARD")
    print("="*70)
    print(f"🕐 {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"📁 Session: {SESSION_ID}")
    print("="*70)
    
    # Sync status
    stats = preserver.sync_now(verbose=False)
    print(f"\n🔄 Auto-Sync Status:")
    print(f"   New files: {stats['new']}")
    print(f"   Updated: {stats['updated']}")
    print(f"   Total synced: {len(preserver.synced_files)}")
    
    # Latest reports
    reports = get_latest_reports()
    print(f"\n📊 Latest Reports:")
    
    if reports['transcripts']:
        latest = reports['transcripts'][0]
        mtime = datetime.fromtimestamp(latest.stat().st_mtime)
        age = (datetime.now() - mtime).total_seconds() / 60
        print(f"   📝 Transcript: {latest.name}")
        print(f"      Updated: {age:.1f} min ago")
    else:
        print(f"   📝 Transcript: None yet")
    
    if reports['summaries']:
        latest = reports['summaries'][0]
        mtime = datetime.fromtimestamp(latest.stat().st_mtime)
        age = (datetime.now() - mtime).total_seconds() / 60
        print(f"   📄 Summary: {latest.name}")
        print(f"      Updated: {age:.1f} min ago")
    else:
        print(f"   📄 Summary: None yet")
    
    if reports['actions']:
        latest = reports['actions'][0]
        try:
            with open(latest, 'r') as f:
                actions = json.load(f)
            print(f"   ⚡ Actions: {len(actions)} items")
        except:
            print(f"   ⚡ Actions: {latest.name}")
    else:
        print(f"   ⚡ Actions: None yet")
    
    # Trajectory
    print(f"\n📈 Trajectory:")
    print(f"   Total artifacts: {len(preserver.trajectory_log)}")
    if preserver.trajectory_log:
        first = preserver.trajectory_log[0]['timestamp']
        last = preserver.trajectory_log[-1]['timestamp']
        print(f"   First: {first.split('T')[1][:8]}")
        print(f"   Latest: {last.split('T')[1][:8]}")
    
    # System state
    state_file = PROJECT_ROOT / "multi-agent/state/deployment_state.json"
    if state_file.exists():
        with open(state_file, 'r') as f:
            state = json.load(f)
        print(f"\n🚀 Deployment:")
        print(f"   Version: {state.get('current_version', 'N/A')}")
        print(f"   Stage: {state.get('stage', 'N/A')}")
    
    print("\n" + "="*70)
    print("🔄 Refreshing in 30 seconds...")
    print("⏹️  Press Stop button to exit")
    print("="*70)

# Run dashboard
print("👁️ Starting live dashboard (Press Stop to exit)\n")

try:
    while True:
        display_dashboard()
        time.sleep(30)
except KeyboardInterrupt:
    print("\n⏹️ Dashboard stopped")

## 📊 Step 9: View Trajectory Summary

In [ ]:
# Generate and display trajectory summary
summary = preserver.create_trajectory_summary()
print(summary)

print(f"\n✅ Trajectory summary saved to:")
print(f"   {DRIVE_TRAJECTORIES / 'trajectory_summary.md'}")

## 📥 Step 10: View Latest Meeting Transcript

In [ ]:
# Read and display latest transcript from Drive
transcripts = sorted(DRIVE_REPORTS.glob("transcript_*.md"), key=lambda x: x.stat().st_mtime, reverse=True)

if transcripts:
    latest = transcripts[0]
    print(f"📝 Latest Transcript: {latest.name}\n")
    print("="*70)
    
    with open(latest, 'r') as f:
        content = f.read()
    
    # Show first 2000 characters
    if len(content) > 2000:
        print(content[:2000])
        print("\n... (truncated)")
        print(f"\n📄 Full transcript: {latest}")
    else:
        print(content)
else:
    print("⏳ No transcripts yet. Wait for first meeting to complete.")

## 🛑 Step 11: Stop System & Final Sync

In [ ]:
print("🛑 Stopping autonomous system...\n")

# Stop coordinator
coordinator.stop()
print("✅ Coordinator stopped")

# Final sync
print("\n🔄 Final sync to Drive...")
stats = preserver.sync_now(verbose=True)
print(f"\n✅ Final sync complete:")
print(f"   New: {stats['new']}")
print(f"   Updated: {stats['updated']}")

# Stop auto-sync
preserver.stop_auto_sync()

# Generate final trajectory
print("\n📊 Generating final trajectory summary...")
summary = preserver.create_trajectory_summary()
print("✅ Trajectory summary saved")

# Create session summary
session_summary = {
    'session_id': SESSION_ID,
    'start_time': preserver.trajectory_log[0]['timestamp'] if preserver.trajectory_log else None,
    'end_time': datetime.now().isoformat(),
    'total_artifacts': len(preserver.trajectory_log),
    'total_synced': len(preserver.synced_files),
    'location': str(DRIVE_SESSION)
}

summary_file = DRIVE_SESSION / "session_summary.json"
with open(summary_file, 'w') as f:
    json.dump(session_summary, f, indent=2)

print("\n" + "="*70)
print("✅ SYSTEM STOPPED - ALL DATA SAVED")
print("="*70)
print(f"\n📁 All files saved to:")
print(f"   Reports: {DRIVE_REPORTS}")
print(f"   Trajectory: {DRIVE_TRAJECTORIES}")
print(f"   Session: {DRIVE_SESSION}")
print(f"\n✅ Safe to disconnect from Colab")

## 🔧 Utility: Manual Sync

In [ ]:
# Run this cell anytime to manually sync
print("🔄 Manual sync...\n")
stats = preserver.sync_now(verbose=True)
print(f"\n✅ Sync complete:")
print(f"   New: {stats['new']}")
print(f"   Updated: {stats['updated']}")
print(f"   Skipped: {stats['skipped']}")

## 🔧 Utility: List All Reports

In [ ]:
# List all reports on Drive
print("📋 All Reports on Google Drive:\n")

for file in sorted(DRIVE_REPORTS.glob("**/*"), key=lambda x: x.stat().st_mtime, reverse=True):
    if file.is_file():
        size = file.stat().st_size / 1024
        mtime = datetime.fromtimestamp(file.stat().st_mtime)
        rel_path = file.relative_to(DRIVE_REPORTS)
        print(f"   {rel_path} ({size:.1f} KB) - {mtime.strftime('%H:%M:%S')}")

---

## 📚 Documentation

### What Gets Saved

**Real-time Reports (auto-synced every 10s):**
- `transcript_*.md` - Complete meeting conversation
- `summary_*.md` - Meeting summary and decisions
- `actions_*.json` - Action items for execution team
- `responses_*.json` - Individual agent responses
- `integrity_*.json` - Integrity checks

**Trajectory Log:**
- `trajectory_log.json` - Complete artifact timeline
- `trajectory_summary.md` - Human-readable summary

**Session Data:**
- `session_summary.json` - Session metadata
- `state/` - System state snapshots

### Recovery from Disconnect

If Colab disconnects:
1. All reports are already on Drive (auto-synced every 10s)
2. Trajectory log preserved in `sessions/session_*/trajectories/`
3. Latest state in `multi-agent/reports/`
4. Re-run notebook to continue from last state

### Monitoring

Use **Step 8** (Live Dashboard) to monitor:
- Real-time sync status
- Latest reports and timestamps
- Trajectory growth
- System deployment state

---

**Version:** 3.0  
**Features:** Auto-Sync ✅ | Trajectories ✅ | Monitoring ✅ | Crash Recovery ✅
